# Task

### Create a Custom Console Logger

Create a context manager called `ConsoleLogger` that:

1. Redirects all `print` statements within its context to a file
2. Restores normal console output when exiting the context
3. Handles any exceptions that occur and logs them to the file
4. Provides a summary of how many lines were written when exiting

Requirements:

- Use a class-based approach implementing __enter__ and __exit__
- The file name should be provided when creating the context manager
- All prints within the context should go to the file instead of console
- Handle at least 2 exception errors, log them to the file instead of showing
- When exiting, print a summary to the console showing how many lines were logged

Example usage should look like this:
```python
with ConsoleLogger('output.log'):
    print("This should go to the file")
    print("This too")
    # If any error occurs, it should be logged to the file
    x = 2/0 # This should log with 'division by zero' message or something similar.
```

Inside the `output.log`:
```txt
This should go to the file
This too
zero division error
```

In [1]:
import sys
from typing import Optional, TextIO

class ConsoleLogger:
    def __init__(self, filename: str):
        self.filename = filename
        self.file: Optional[TextIO] = None
        self.original_stdout = None
        self.lines_logged = 0

    def __enter__(self):
        self.file = open(self.filename, "w", encoding="utf-8")
        self.original_stdout = sys.stdout
        sys.stdout = self
        return self

    def write(self, message: str) -> int:
        if self.file is None:
            return 0
        n = self.file.write(message)
        self.file.flush()
        self.lines_logged += message.count("\n")
        return n

    def flush(self):
        if self.file is not None:
            self.file.flush()

    def __exit__(self, exc_type, exc, tb):
        suppress = False

        if self.file is not None and exc_type is not None:
            if isinstance(exc, ZeroDivisionError):
                self.file.write("zero division error\n")
                self.lines_logged += 1
                suppress = True
            elif isinstance(exc, ValueError):
                self.file.write("value error\n")
                self.lines_logged += 1
                suppress = True
            else:
                self.file.write(f"unexpected error: {exc}\n")
                self.lines_logged += 1

        sys.stdout = self.original_stdout

        if self.file is not None:
            self.file.close()

        print(f"ConsoleLogger: logged {self.lines_logged} line(s) to '{self.filename}'")

        return suppress


In [3]:
with ConsoleLogger('output.log'):
    print("This should go to the file")
    print("This too")
    x = 2 / 0

ConsoleLogger: logged 3 line(s) to 'output.log'
